[![Fixel Algorithms](https://fixelalgorithms.co/images/CCExt.png)](https://fixelalgorithms.gitlab.io/)

# Machine Learning Methods

## Supervised Learning - Classification Performance Scores / Metrics: Precision, Recall, ROC and AUC - Exercise

> Notebook by:
> - Royi Avital RoyiAvital@fixelalgorithms.com

## Revision History

| Version | Date       | User        |Content / Changes                                                   |
|---------|------------|-------------|--------------------------------------------------------------------|
| 0.1.000 | 23/01/2023 | Royi Avital | First version                                                      |
|         |            |             |                                                                    |

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/FixelAlgorithmsTeam/FixelCourses/blob/master/MachineLearningMethods/2023_01/0014PerformanceScoreMetricsExerciseSolution.ipynb)

In [ ]:
# Import Packages

# General Tools
import numpy as np
import scipy as sp
import pandas as pd

# Machine Learning
from sklearn.datasets import make_moons
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import auc, confusion_matrix, precision_recall_fscore_support, roc_curve
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

# Miscellaneous
import os
from platform import python_version
import random

# Typing
from typing import Tuple

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
from bokeh.plotting import figure, show

# Jupyter
from IPython import get_ipython
from IPython.display import Image, display
from ipywidgets import Dropdown, FloatSlider, interact, IntSlider, Layout

## Notations

* <font color='red'>(**?**)</font> Question to answer interactively.
* <font color='blue'>(**!**)</font> Simple task to add code for the notebook.
* <font color='green'>(**@**)</font> Optional / Extra self practice.
* <font color='brown'>(**#**)</font> Note / Useful resource / Food for thought.

In [ ]:
# Configuration
%matplotlib inline

seedNum = 512
np.random.seed(seedNum)
random.seed(seedNum)

# sns.set_theme() #>! Apply SeaBorn theme

runInGoogleColab = 'google.colab' in str(get_ipython())

In [ ]:
# Constants

FIG_SIZE_DEF = (8, 8)
ELM_SIZE_DEF = 50
CLASS_COLOR = ('b', 'r')
EDGE_COLOR  = 'k'


In [ ]:
# Fixel Algorithms Packages


## Calibrating the Model Performance

In this exercise we'll learn few approaches dealing wiht imbalnced data dn tuning performance:

 - Resampling.
 - Weighing (Class / Samples).
 - Probability Threshold.

We'll do that using the SVM model, though they generalize to most models.

* <font color='brown'>(**#**)</font> Pay attention that in order to have the probability per class on the _SVC_ class we need to set `probability = True`.
* <font color='brown'>(**#**)</font> The process of `probability = True` is not always consistent with the `decision_function()` method. Hence it is better to use it in the case of the `SVC`.
* <font color='brown'>(**#**)</font> In the above, all approaches are during the training time. One could also search for the best model, score wise, using _Cross Validation_.


In [ ]:
# Parameters

# Data Generation
numSamples0 = 950
numSamples1 = 50

noiseLevel = 0.1

testSize = 0.5

# Model
paramC      = 1
kernelType  = 'linear'

# Data Visualization
numGridPts = 500

In [ ]:
# Auxiliary Functions

def PlotBinaryClassData( mX: np.ndarray, vY: np.ndarray, hA:plt.Axes = None, figSize: Tuple[int, int] = FIG_SIZE_DEF, elmSize: int = ELM_SIZE_DEF, classColor: Tuple[str, str] = CLASS_COLOR, axisTitle: str = None ) -> plt.Axes:

    if hA is None:
        hF, hA = plt.subplots(figsize = figSize)
    else:
        hF = hA.get_figure()
    
    vC, vN = np.unique(vY, return_counts = True)

    numClass = len(vC)
    if (len(vC) != 2):
        raise ValueError(f'The input data is not binary, the number of classes is: {numClass}')

    vIdx0 = vY == vC[0]
    vIdx1 = vY == vC[1] #<! Basically ~vIdx0

    hA.scatter(mX[vIdx0, 0], mX[vIdx0, 1], s = elmSize, color = classColor[0], edgecolor = 'k', label = f'$C_\u007b {vC[0]} \u007d$')
    hA.scatter(mX[vIdx1, 0], mX[vIdx1, 1], s = elmSize, color = classColor[1], edgecolor = 'k', label = f'$C_\u007b {vC[1]} \u007d$')
    hA.axvline(x = 0, color = 'k')
    hA.axhline(y = 0, color = 'k')
    hA.axis('equal')
    if axisTitle is not None:
        hA.set_title(axisTitle)
    hA.legend()
    
    return hA

def PlotLabelsHistogram(vY: np.ndarray, hA = None):

    if hA is None:
        hF, hA = plt.subplots(figsize = (8, 6))
    
    vLabels, vCounts = np.unique(vY, return_counts = True)

    hA.bar(vLabels, vCounts, width = 0.9, align = 'center')
    hA.set_xticks(vLabels)
    hA.set_title('Histogram of Classes / Labels')
    hA.set_xlabel('Class')
    hA.set_ylabel('Number of Samples')

    return hA

def PlotConfusionMatrix(vY: np.ndarray, vYPred: np.ndarray, hA: plt.Axes = None, lLabels: list = None, dScore: dict = None, titleStr: str = 'Confusion Matrix') -> plt.Axes:

    # Calculation of Confusion Matrix
    mConfMat = confusion_matrix(vY, vYPred)
    oConfMat = ConfusionMatrixDisplay(mConfMat, display_labels = lLabels)
    oConfMat = oConfMat.plot(ax = hA)
    hA = oConfMat.ax_
    if dScore is not None:
        titleStr += ':'
        for scoreName, scoreVal in  dScore.items():
            titleStr += f' {scoreName} = {scoreVal:0.2},'
        titleStr = titleStr[:-1]
    hA.set_title(titleStr)
    hA.grid(False)

    return hA


def PlotDecisionBoundaryClosure( numGridPts, gridXMin, gridXMax, gridYMin, gridYMax, numDigits = 1 ):

    # v0       = np.linspace(gridXMin, gridXMax, numGridPts)
    # v1       = np.linspace(gridYMin, gridYMax, numGridPts)
    roundFctr = 10 ** numDigits
    
    # For equal axis
    minVal = np.floor(roundFctr * min(gridXMin, gridYMin)) / roundFctr
    maxVal = np.ceil(roundFctr * max(gridXMax, gridYMax)) / roundFctr
    v0     = np.linspace(minVal, maxVal, numGridPts)
    v1     = np.linspace(minVal, maxVal, numGridPts)
    
    XX0, XX1 = np.meshgrid(v0, v1)
    XX       = np.c_[XX0.ravel(), XX1.ravel()]

    def PlotDecisionBoundary(hDecFun, hA = None):
        
        if hA is None:
            hF, hA = plt.subplots(figsize = (8, 6))

        Z = hDecFun(XX)
        Z = Z.reshape(XX0.shape)
            
        hA.contourf(XX0, XX1, Z, colors = CLASS_COLOR, alpha = 0.3, levels = [-0.5, 0.5, 1.5])

        return hA

    return PlotDecisionBoundary
    




## Generate / Load Data


In [ ]:
# Loading / Generating Data

mX, vY = make_moons(n_samples = [numSamples0, numSamples1], noise = noiseLevel)

print(f'The features data shape: {mX.shape}')
print(f'The labels data shape: {vY.shape}')
print(f'The unique values of the labels: {np.unique(vY)}')

### Plot Data

In [ ]:
# Display the Data

vIdx0 = vY == 0
vIdx1 = vY == 1

mX0 = mX[vIdx0]
mX1 = mX[vIdx1]

hA = PlotBinaryClassData(mX, vY, axisTitle = 'Samples Data')

### Distribution of Labels

When dealing with classification, it is important to know the balance between the labels within the data set.

In [ ]:
hA = PlotLabelsHistogram(vY)
plt.show()

## Train SVM Classifier

In [ ]:
# SVM Linear Model
# Train a model and set the parameter `probability` to `True`
#===========================Fill This===========================#
oSVM  = SVC(C = paramC, kernel = kernelType, probability = ???).fit(mX, vY) #<! We can do the training in a one liner
#===============================================================#

modelScore = oSVM.score(mX, vY)

print(f'The model score (Accuracy) on the data: {modelScore}') #<! Accuracy

### Plot Decision Boundary

We'll display, the linear, decision boundary of the classifier.

In [ ]:
# Decision Boundary Plotter (Per Data!)
# Look at the implemetation for an example for a Closure in Python

PlotDecisionBoundary = PlotDecisionBoundaryClosure(numGridPts, mX[:, 0].min(), mX[:, 0].max(), mX[:, 1].min(), mX[:, 1].max())


In [ ]:
# Plotting the decision boundary
hF, hA = plt.subplots(figsize = FIG_SIZE_DEF)
hA = PlotDecisionBoundary(oSVM.predict, hA)
hA = PlotBinaryClassData(mX, vY, hA = hA, axisTitle = 'Classifier Decision Boundary')
plt.show()

In [ ]:
# Extract confidence level in the prediction

#===========================Fill This===========================#
vD = ??? #<! Apply the decision function of the data set
#===============================================================#

mP = oSVM.predict_proba(mX) #<! Probabilities per class

In [ ]:
# The built in probability doesn't match the decision function of the classifier!

hF, hA = plt.subplots(figsize = FIG_SIZE_DEF)
vSampleIdx = list(range(1, mX.shape[0] + 1))

hA.scatter(vSampleIdx, vD > 0, s = 3 * ELM_SIZE_DEF, label = 'Class by Decision Function')
hA.scatter(vSampleIdx, np.argmax(mP, axis = 1), s = ELM_SIZE_DEF, label = 'Class by Probability')
hA.set_xlabel('Sample Index')
hA.set_ylabel('Predicted Class')
hA.legend()

plt.show()

<font color='red'>(**?**)</font> Which one matches the trained model?

In [ ]:
# Probability function for Binary SVM Classifier
# Idea is to create function with asymptotic behaviour
# The input is the per sample output of `decision_function()` method

def SvcBinProb(vD):
    mP = np.zeros(shape = (vD.shape[0], 2))

    mP[:, 1] = 0.5 * (1 + np.sign(vD) * (1 - np.exp(-np.abs(vD)))) #>! The probability of the positive class
    mP[:, 0] = 1 - mP[:, 1]

    return mP

In [ ]:
# Probability per Sample per Class
# Calculate the probability matrix using `SvcBinProb`

#===========================Fill This===========================#
mP = ???
#===============================================================#

In [ ]:
# Verify visually that the classification by `mP` and `vD` match

hF, hA = plt.subplots(figsize = FIG_SIZE_DEF)
vSampleIdx = list(range(1, mX.shape[0] + 1))

hA.scatter(vSampleIdx, vD > 0, s = 3 * ELM_SIZE_DEF, label = 'Class by Decision Function')
hA.scatter(vSampleIdx, np.argmax(mP, axis = 1), s = ELM_SIZE_DEF, label = 'Class by Probability')
hA.set_xlabel('Sample Index')
hA.set_ylabel('Predicted Class')
hA.legend()

plt.show()

In [ ]:
# Check programmitically that the classification by `mP` and `vD` match

#===========================Fill This===========================#
???
#===============================================================#


## Tuning Metrics / Scores

In this section we'll show the effect of the tuning on the performance and the decision boundary.

### Probability Threshold Tuning

Most classifiers have a threshold based decision rule for binary classification.  
In many cases it is based on probability on others (Such as in the SVM) on a different confidence function.  

In the aboce we transformed the confidence function of the SVM into probability.  
Now, we'll use the probability threshold as a way to play with the _working point_ of the classifier.

In [ ]:
# Extract the probability of the positive class
# The array `mP` has the probability for both classes
# Extract from it the probability of Class = 1

#===========================Fill This===========================#
vP = ???
#===============================================================#

vFP, vTP, vThr = roc_curve(vY, vP, pos_label = 1)
AUC            = auc(vFP, vTP)

In [ ]:
# Plotting Function

hDecFunc = lambda XX, probThr: SvcBinProb(oSVM.decision_function(XX))[:, 1].reshape((numGridPts, numGridPts)) > probThr

def PlotRoc(probThr):
    _, vAx = plt.subplots(1, 2, figsize = (14, 6))
    hA = vAx[0]
    hA.plot(vFP, vTP, color = 'b', lw = 3, label = f'AUC = {AUC:.3f}')
    hA.plot([0, 1], [0, 1], color = 'k', lw = 2, linestyle = '--')

    vIdx = np.flatnonzero(vThr < probThr)
    if vIdx.size == 0:
        idx = -1
    else:
        idx = vIdx[0] - 1

    hA.axvline(x = vFP[idx], color = 'g', lw = 2, linestyle = '--')
    hA.set_xlabel('False Positive Rate')
    hA.set_ylabel('True Positive Rate')
    hA.set_title ('ROC')
    hA.axis('equal')
    hA.legend()
    hA.grid()    
    
    hA = vAx[1]

    hA = PlotDecisionBoundary(lambda XX: hDecFunc(XX, probThr), hA)
    hA = PlotBinaryClassData(mX, vY, hA = hA)
    


In [ ]:
probThrSlider = FloatSlider(min = 0.0, max = 1.0, step = 0.01, value = 0.5, readout_format = '0.2%', layout = Layout(width = '30%'))
interact(PlotRoc, probThr = probThrSlider)

plt.show()

### Resampling

Another way to have a better default classifier for the imbalanced data is to resample the data in a balanced way:

![](https://i.imgur.com/kPGo65I.png)

* <font color='brown'>(**#**)</font> There is a dedicated Python package for that called [`imbalanced-learn`](https://github.com/scikit-learn-contrib/imbalanced-learn) which automates this.  
  It also uses some more advanced tricks. Hence in practice, if you chose resampling approach, use it.
* <font color='brown'>(**#**)</font> While in the following example we'll resample the whole data set, in practive we'll do resampling only on the training data set.  
  This is in order to avoid _data leakage_.

#### Undersampling

In case we have enough samples to learn from in the smaller class, this is the way to go.

In [ ]:
# Under Sample the Class 0 Samples
# Using Numpy `choice()` we can resample indices with or without replacement.
# In this case we need to undersample, hence with no replacement.

#===========================Fill This===========================#
vIdx0UnderSample = np.random.choice(???, ???, replace = ???)
mX0UnderSample   = ???
#===============================================================#

mXS = np.vstack((mX0UnderSample, mX1))
vYS = np.concatenate((np.zeros(mX0UnderSample.shape[0], dtype = vY.dtype), np.ones(mX1.shape[0], dtype = vY.dtype)))

In [ ]:
# Plot resmapled data
hA = PlotBinaryClassData(mXS, vYS, axisTitle = 'Resampled Data')

In [ ]:
# SVM Linear Model
oSVM  = SVC(C = paramC, kernel = kernelType).fit(mXS, vYS) #<! Fit on the new sampled data

In [ ]:
# Plot decision boundary
hF, hA = plt.subplots(figsize = FIG_SIZE_DEF)
hA = PlotDecisionBoundary(oSVM.predict, hA)
hA = PlotBinaryClassData(mXS, vYS, hA = hA, axisTitle = 'Classifier Decision Boundary - Resampled Data')
plt.show()

#### Oversampling

In this case we resample the less populated class to have similar number of samples as the more populated class.

In [ ]:
# Over Sample the Class 1 Samples
# In this case we'll utilize `choice()` with replacement

#===========================Fill This===========================#
numSamplesGenerate = ??? #<! Number of samples to generate
vIdx1OverSample = np.random.choice(???, numSamplesGenerate replace = ???)
mX1OverSampleSample = ???
#===============================================================#

mXS = np.vstack((mX0, mX1OverSampleSample))
vYS = np.concatenate((np.zeros(mX0.shape[0], dtype = vY.dtype), np.ones(mX1OverSampleSample.shape[0], dtype = vY.dtype)))


In [ ]:
# Plot resmapled data
hA = PlotBinaryClassData(mXS, vYS, axisTitle = 'Resampled Data')
plt.show()

In [ ]:
# SVM Linear Model
oSVM  = SVC(C = paramC, kernel = kernelType).fit(mXS, vYS) #<! Fir on teh new sampled data

In [ ]:
# Plot decision boundary
hF, hA = plt.subplots(figsize = FIG_SIZE_DEF)
hA = PlotDecisionBoundary(oSVM.predict, hA)
hA = PlotBinaryClassData(mXS, vYS, hA = hA, axisTitle = 'Classifier Decision Boundary - Resampled Data')
plt.show()

* <font color='red'>(**?**)</font> Is the result the same as in the _under sample_ method? Why?

### Data Weighing

Another approach is changing the weights of the data.  
We have 2 methods here:

 - Weight per Sample  
   This is very similar in effect to resample the sample. The difference is having ability for non integer weight.  
   It may not be available in all classifiers on SciKit Learn (For example, `SVC` doesn't support this).
 - Weight per Class  
   Applies the weighing on the samples according to their class.   
   Usually applied in SciKit Learn under `class_weight`.
   It has a `balanced` option which tries to balance imbalanced data.

In [ ]:
# SVM Linear Model - Balanced weighing

#===========================Fill This===========================#
oSVM  = SVC(C = paramC, kernel = kernelType, class_weight = ???).fit(mX, vY)
#===============================================================#

In [ ]:
# Plot decision boundary
hF, hA = plt.subplots(figsize = FIG_SIZE_DEF)
hA = PlotDecisionBoundary(oSVM.predict, hA)
hA = PlotBinaryClassData(mX, vY, hA = hA, axisTitle = f'Classifier Decision Boundary: Class 0 Weight = {oSVM.class_weight_[0]:0.2f}, Class 1 Weight = {oSVM.class_weight_[1]:0.2f}')
plt.show()

In [ ]:
# SVM Linear Model - Manual weighing
# We'll set the weight of the class 0 to 1, and class 1 to 1000

#===========================Fill This===========================#
dClassWeight = {???} #<! Fill a dictionary
#===========================Fill This===========================#

oSVM  = SVC(C = paramC, kernel = kernelType, class_weight = dClassWeight).fit(mX, vY)

In [ ]:
# Plot decision boundary
hF, hA = plt.subplots(figsize = FIG_SIZE_DEF)
hA = PlotDecisionBoundary(oSVM.predict, hA)
hA = PlotBinaryClassData(mX, vY, hA = hA, axisTitle = f'Classifier Decision Boundary: Class 0 Weight = {oSVM.class_weight_[0]:0.2f}, Class 1 Weight = {oSVM.class_weight_[1]:0.2f}')
plt.show()